# Labels

> Fill in a module description here

In [ ]:
#| default_exp labels

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import tifffile
from cellpose import models, io
import matplotlib.pyplot as plt
from typing import Tuple
from skimage.segmentation import find_boundaries
from scipy.ndimage import distance_transform_edt
from imageio import imwrite


def predict(
    model_path: str,
    img_list: list,
    use_gpu: bool,
    cellpose_param: tuple
) -> np.ndarray:
    """
    Run a pretrained Cellpose model to generate segmentation masks for a list of images.

    Parameters
    ----------
    model_path : str
        Path to the pretrained Cellpose model file.
    img_list : iterable
        Iterable of image arrays to segment.
    cellpose_param : tuple of float
        Tuple specifying (flow_threshold, cellprob_threshold) for Cellpose evaluation.

    Returns
    -------
    mask : np.ndarray
        Array of shape (N, H, W) containing the segmentation masks for each input image.
    """
    model_ft = models.CellposeModel(
        gpu=use_gpu,
        pretrained_model=model_path
    )
    mask, flow, style = model_ft.eval(
        list(img_list),
        flow_threshold=cellpose_param[0],
        cellprob_threshold=cellpose_param[1],
        channels=[0, 0]
    )
    return mask


def plot_outlines(
    mask: np.ndarray,
    image: np.ndarray,
    save_path: str = None,
    ax: plt.Axes = None
):
    """
    Overlay nuclear outlines from a mask onto an image and optionally save the result.

    Parameters
    ----------
    mask : np.ndarray
        2D array of integer labels where nonzero values indicate nuclei.
    image : np.ndarray
        Grayscale image array of the same shape as `mask`.
    save_path : str, optional
        File path in which to save the outline image. If None, no file is written.

    Returns
    -------
    fig : matplotlib.figure.Figure
        Figure object containing the outline overlay.
    ax : matplotlib.axes.Axes
        Axes object displaying the overlay.
    """
    boundaries = find_boundaries(mask, mode='outer')
    outline_image = np.zeros_like(mask, dtype=np.uint8)
    outline_image[boundaries] = 255

    h, w = mask.shape
    green = np.zeros((h, w), dtype=np.uint8)
    stacked = np.stack([outline_image, green, image], axis=-1)

    if save_path:
        imwrite(save_path, stacked)

    return stacked


def get_physical_size(
    filename: str,
    verbose: bool = False) -> float:
    """
    Compute the physical pixel size (µm/px) from a TIFF file's resolution metadata.

    Parameters
    ----------
    filename : str
        Path to the TIFF file.
    verbose : bool, optional
        If True, print resolution details. Default is False.

    Returns
    -------
    px_um : float
        Physical size of a pixel in microns. Raises if pixels are non-square.
    """

    with tifffile.TiffFile(filename) as tif:
        page = tif.pages[0]

        rx_num, rx_den = page.tags['XResolution'].value
        ry_num, ry_den = page.tags['YResolution'].value
        unit_code = page.tags['ResolutionUnit'].value

        if unit_code == 2:
            um_per_unit = 25400.0
        elif unit_code == 3:
            um_per_unit = 10000.0
        else:
            raise ValueError(f"Unexpected ResolutionUnit {unit_code}")

        px_per_unit_x = rx_num / rx_den
        px_per_unit_y = ry_num / ry_den

        px_w_um = um_per_unit / px_per_unit_x
        px_h_um = um_per_unit / px_per_unit_y

        width = page.imagewidth
        height = page.imagelength

        if verbose:
            #print(f"ResolutionUnit code: {unit_code}")
            print(f"Pixel size = {px_w_um:.3f} × {px_h_um:.3f} µm/px")
            print(f"Physical size = {width * px_w_um:.1f} × {height * px_h_um:.1f} µm")

        if px_h_um == px_w_um:
            return px_h_um
        else:
            raise ValueError("Pixel height and width are not the same")


def expand_with_cap(
    label_image: np.ndarray,
    spacing: Tuple[float, float],
    fixed_expand: float = 10.0,
    max_area_ratio: float = 1.5
) -> np.ndarray:
    """
    Expand labeled nuclei by up to a fixed radius, capped by max area ratio.

    Parameters
    ----------
    label_image : np.ndarray
        2D integer array of labels (0=background).
    spacing : tuple of float
        Physical size (µm) of each pixel along (row, column) axes.
    fixed_expand : float, optional
        Nominal expansion radius in µm. Default is 10.0.
    max_area_ratio : float, optional
        Maximum allowed area increase factor. Default is 1.5.

    Returns
    -------
    labels_out : np.ndarray
        Dilated label image, where each pixel is assigned to the nearest
        original label if within its per-pixel cap.
    """
    pixel_area = np.prod(spacing)
    labels = np.unique(label_image)
    labels = labels[labels != 0]
    r0 = {lbl: np.sqrt((np.sum(label_image == lbl) * pixel_area) / np.pi)
          for lbl in labels}

    factor = np.sqrt(max_area_ratio) - 1.0
    d_max = {lbl: factor * r0[lbl] for lbl in labels}

    max_lbl = int(label_image.max())
    d_max_arr = np.zeros(max_lbl + 1, dtype=float)
    for lbl, dm in d_max.items():
        d_max_arr[lbl] = dm

    distances, nearest_coords = distance_transform_edt(
        label_image == 0,
        sampling=spacing,
        return_indices=True
    )
    nearest_lbls = label_image[tuple(nearest_coords)]

    per_pixel_cap = np.minimum(fixed_expand, d_max_arr[nearest_lbls])
    dilate_mask = distances <= per_pixel_cap

    labels_out = np.zeros_like(label_image)
    labels_out[dilate_mask] = nearest_lbls[dilate_mask]
    
    return labels_out